# Testing the prioritization schemes

## Import Libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import joblib

## Getting the Data

In [2]:
data = pd.read_csv('fake_CRM_data\placements.csv')
data.head(5)

<>:1: SyntaxWarning: "\p" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\p"? A raw string is also an option.
<>:1: SyntaxWarning: "\p" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\p"? A raw string is also an option.
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11560\186369906.py:1: SyntaxWarning: "\p" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\p"? A raw string is also an option.
  data = pd.read_csv('fake_CRM_data\placements.csv')


,Client,Placement Client Local ID,Placement Name,Coverage,Product Line,Carrier Group,Placement Created Date/Time,Placement Created By,Placement Created By (ID),Response Received Date,Placement Specialist,Placement Renewing Status,Placement Status,Declination Reason,Placement Id,Placement Effective Date,Placement Expiry Date,Incumbent Indicator,Participation Status Code,Placement Client Segment Code,Placement Renewing Status Code,Limit,Coverage Premium Amount,Tria Premium,Total Premium,Comission %,Comission Amount,Participation Percentage,Carrier Group Local ID,Production Code,Submission Sent Date,Program Product Local Code Text,Approach Non Admitted Market Indicator,Carrier Integration
0,Global Technologies,SCR-0b810b6f4c20,SCR-8d9f15ee3a3c,General Liability,Energy and Power,Eastern Risk Management,2025-04-24T06:37:09.314837765,Kimberly Jackson,SCR-c54656cdfecb,29/07/25,Mary Jackson,In progress,Quote,-,SCR-76fd0b40a1cb,30/09/25,30/09/26,N,QUOTATION_STATUS_QUOTED,CLIENT_SEGMENT_RISK_MGMT,RENEWAL_STATUS_IN_PROGRESS,3558700.0,65304.28,1881.62,67311.79,8,5760.41,100,0498,PRODUCTION_TYPE_NEW,-,SCR-262eac00ad8f,N,Not Applicable
1,Apex Enterprises,SCR-cfcd65ae0f7a,SCR-6b2e08c3f11d,General Liability,Casualty,Liberty Insurance Group,2025-04-25T15:21:40.575782415,Matthew Johnson,SCR-d47a7dd7c8ff,29/07/25,Donald Martin,In progress,Quote,-,SCR-07c81694713d,15/09/25,15/09/26,Y,QUOTATION_STATUS_QUOTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,82254400.0,1441557.71,55830.17,1581997.56,15,282637.79,100,0815,PRODUCTION_TYPE_RENEWAL,-,SCR-af66d3bd1f7b,Y,Not Applicable
2,Omega Global,SCR-51dd14cf0f45,SCR-6d2b42de057e,General Liability,Casualty,United Coverage Corp,2025-03-13T18:03:34.574937777,Robert Young,SCR-0b51d24e469e,-,Michelle Anderson,In progress,Submitted,-,SCR-b422393095aa,15/09/25,15/09/26,Y,QUOTATION_STATUS_SUBMITTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,225000.0,3279.01,0.00,3256.21,17,645.63,100,3093,PRODUCTION_TYPE_RENEWAL,29/07/25,SCR-af66d3bd1f7b,N,Not Applicable
3,Omega Global,SCR-51dd14cf0f45,SCR-6d2b42de057e,General Liability,Casualty,United Coverage Corp,2025-03-13T18:03:34.574937777,Robert Young,SCR-0b51d24e469e,-,Michelle Anderson,In progress,Submitted,-,SCR-b422393095aa,15/09/25,15/09/26,Y,QUOTATION_STATUS_SUBMITTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,225000.0,0.00,0.00,0.00,17,0.00,100,3093,PRODUCTION_TYPE_RENEWAL,29/07/25,SCR-af66d3bd1f7b,N,Not Applicable
4,Omega Global,SCR-51dd14cf0f45,SCR-6d2b42de057e,General Liability,Casualty,Northern Trust Insurance,2025-03-13T18:03:34.574937777,Robert Young,SCR-0b51d24e469e,-,Michelle Anderson,In progress,Submitted,-,SCR-b422393095aa,15/09/25,15/09/26,N,QUOTATION_STATUS_SUBMITTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,225000.0,0.00,0.00,0.00,17,0.00,100,4584,PRODUCTION_TYPE_RENEWAL,30/07/25,SCR-af66d3bd1f7b,N,Not Applicable


## Preprocessing

In [3]:
def camelCase(names: list) -> list:
  return [name\
          .strip() \
          .replace('_', ' ') \
        #   .title() \
          .replace(' ', '') \
          .replace('(', '_') \
          .replace(')', '') \
          for name in names]

data.columns = camelCase(data.columns)
data.rename({'PlacementCreatedDate/Time': 'PlacemenCreatedDatetime'}, inplace=True)

In [4]:
for col in data.columns:
    data[col] = data[col].replace('-', pd.NA)

In [5]:
def setColsToDatetime(df: pd.DataFrame, cols: list) -> None:
    for col in cols:
        df[col] = pd.to_datetime(df[col], dayfirst=True, errors='ignore')

In [6]:
setColsToDatetime(
    df=data, 
    cols=\
        ['ResponseReceivedDate', 'PlacementEffectiveDate',
       'PlacementExpiryDate', 'SubmissionSentDate']
)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11560\3892670953.py:3: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_datetime(df[col], dayfirst=True, errors='ignore')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11560\3892670953.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], dayfirst=True, errors='ignore')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11560\3892670953.py:3: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df[col] = pd.to_datetime(df[col], dayfirst=True, errors='ignore')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11560\3892670953.py:3: UserWarning: Cou

### Adding new columns

In [7]:
data['_DaysToExpiry'] = (data.PlacementExpiryDate - data.PlacementEffectiveDate).apply(lambda x: x.days)
data['_CarrierResponseTime'] = (data.ResponseReceivedDate - data.SubmissionSentDate).apply(lambda x: x.days)

In [8]:
clients = data.PlacementClientLocalID.unique()
data['_ChurnStatus'] = pd.Series([pd.NA for _ in range(data.shape[0])], dtype='str')
for client in clients:
    clientMask = (data.PlacementClientLocalID == client)
    years = data[clientMask].PlacementExpiryDate.apply(lambda x: x.year).unique()
    status = ''
    
    for year in years:
        yearMask = (data.PlacementExpiryDate.apply(lambda x: x.year) == year)
        count = data[clientMask & yearMask].shape[0]
        if 'QUOTATION_STATUS_BOUND' in data[clientMask & yearMask].ParticipationStatusCode.to_list():            
            data.loc[clientMask & yearMask, '_ChurnStatus'] = 'N'
        else:
            data.loc[clientMask & yearMask, '_ChurnStatus'] = 'C'

In [9]:
def fracRenewed(group):
    x = group[group == 'N'].size
    y = group.size
    return x/(x+y)

fracPlacementsRenewedByClient = data.groupby(['PlacementClientLocalID'])['_ChurnStatus'].agg(fracRenewed)
# fracPlacementsRenewedByClient = fracPlacementsRenewedByClient.rename({'PlacementRenewingStatusCode':'FracPlacementsRenewed'})

In [10]:
clients = fracPlacementsRenewedByClient.index.unique()
data['_FracPlacementsRenewedByClient'] = pd.Series([0. for _ in range(data.shape[0])], dtype='float')
for client in clients:
    clientMask = (data.PlacementClientLocalID == client)
    data.loc[clientMask, '_FracPlacementsRenewedByClient'] = fracPlacementsRenewedByClient[client]

In [11]:
def fracRenewed(group):
    x = group[group == 'N'].size
    y = group.size
    return x/(x+y)

fracPlacementsRenewedByCarrier = data.groupby(['CarrierGroupLocalID'])['_ChurnStatus'].agg(fracRenewed)

In [12]:
carriers = fracPlacementsRenewedByCarrier.index.unique()
data['_FracPlacementsRenewedByCarrier'] = pd.Series([0. for _ in range(data.shape[0])], dtype='float')
for carrier in carriers:
    carrierMask = (data.CarrierGroupLocalID == carrier)
    data.loc[carrierMask, '_FracPlacementsRenewedByCarrier'] = fracPlacementsRenewedByCarrier[carrier]

In [13]:
data.head(3)

,Client,PlacementClientLocalID,PlacementName,Coverage,ProductLine,CarrierGroup,PlacementCreatedDate/Time,PlacementCreatedBy,PlacementCreatedBy_ID,ResponseReceivedDate,PlacementSpecialist,PlacementRenewingStatus,PlacementStatus,DeclinationReason,PlacementId,PlacementEffectiveDate,PlacementExpiryDate,IncumbentIndicator,ParticipationStatusCode,PlacementClientSegmentCode,PlacementRenewingStatusCode,Limit,CoveragePremiumAmount,TriaPremium,TotalPremium,Comission%,ComissionAmount,ParticipationPercentage,CarrierGroupLocalID,ProductionCode,SubmissionSentDate,ProgramProductLocalCodeText,ApproachNonAdmittedMarketIndicator,CarrierIntegration,_DaysToExpiry,_CarrierResponseTime,_ChurnStatus,_FracPlacementsRenewedByClient,_FracPlacementsRenewedByCarrier
0,Global Technologies,SCR-0b810b6f4c20,SCR-8d9f15ee3a3c,General Liability,Energy and Power,Eastern Risk Management,2025-04-24T06:37:09.314837765,Kimberly Jackson,SCR-c54656cdfecb,2025-07-29,Mary Jackson,In progress,Quote,<NA>,SCR-76fd0b40a1cb,2025-09-30,2026-09-30,N,QUOTATION_STATUS_QUOTED,CLIENT_SEGMENT_RISK_MGMT,RENEWAL_STATUS_IN_PROGRESS,3558700.0,65304.28,1881.62,67311.79,8,5760.41,100,0498,PRODUCTION_TYPE_NEW,NaT,SCR-262eac00ad8f,N,Not Applicable,365,NaN,C,0.000000,0.477273
1,Apex Enterprises,SCR-cfcd65ae0f7a,SCR-6b2e08c3f11d,General Liability,Casualty,Liberty Insurance Group,2025-04-25T15:21:40.575782415,Matthew Johnson,SCR-d47a7dd7c8ff,2025-07-29,Donald Martin,In progress,Quote,<NA>,SCR-07c81694713d,2025-09-15,2026-09-15,Y,QUOTATION_STATUS_QUOTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,82254400.0,1441557.71,55830.17,1581997.56,15,282637.79,100,0815,PRODUCTION_TYPE_RENEWAL,NaT,SCR-af66d3bd1f7b,Y,Not Applicable,365,NaN,C,0.480000,0.484444
2,Omega Global,SCR-51dd14cf0f45,SCR-6d2b42de057e,General Liability,Casualty,United Coverage Corp,2025-03-13T18:03:34.574937777,Robert Young,SCR-0b51d24e469e,NaT,Michelle Anderson,In progress,Submitted,<NA>,SCR-b422393095aa,2025-09-15,2026-09-15,Y,QUOTATION_STATUS_SUBMITTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,225000.0,3279.01,0.00,3256.21,17,645.63,100,3093,PRODUCTION_TYPE_RENEWAL,2025-07-29,SCR-af66d3bd1f7b,N,Not Applicable,365,NaN,C,0.214286,0.478673


## Testing the Functions

In [14]:
from prioritization import churn_prob_score

churn_prob_score(data)

d:\TejaMeruva\ApexRenew\prioritization.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.PlacementClientSegmentCode.replace(pd.NA, 'null', inplace=True)
d:\TejaMeruva\ApexRenew\prioritization.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.IncumbentIndicator.replace(pd.NA, 'null', inplace=True)


0       0.411721
1       0.756677
2       0.556380
3       0.304154
4       0.344955
          ...   
1343    7.971068
1344    4.087537
1345    5.682493
1346    8.816766
1347    8.816766
Length: 1348, dtype: float64

In [15]:
from prioritization import time_to_expiry_score

time_to_expiry_score(data._DaysToExpiry)

0       6.465134
1       6.465134
2       6.465134
3       6.465134
4       6.465134
          ...   
1343    0.211424
1344    0.211424
1345    0.081602
1346    0.114985
1347    0.114985
Name: _DaysToExpiry, Length: 1348, dtype: float64

In [16]:
from prioritization import premium_at_risk_score

premium_at_risk_score(data.TotalPremium)

0       7.047478
1       9.777448
2       3.531157
3       1.123887
4       1.123887
          ...   
1343    4.339763
1344    4.272997
1345    9.406528
1346    5.871662
1347    5.871662
Name: TotalPremium, Length: 1348, dtype: float64

In [17]:
from prioritization import past_performance_score
past_performance_score(data._FracPlacementsRenewedByCarrier)

0       2.830119
1       5.300445
2       3.816766
3       3.816766
4       1.012611
          ...   
1343    9.321217
1344    4.751484
1345    2.240356
1346    5.300445
1347    5.300445
Name: _FracPlacementsRenewedByCarrier, Length: 1348, dtype: float64

In [18]:
from prioritization import past_performance_score
past_performance_score(data._FracPlacementsRenewedByClient)

0       0.226261
1       1.320475
2       0.671365
3       0.671365
4       0.671365
          ...   
1343    5.686202
1344    5.686202
1345    5.686202
1346    5.686202
1347    5.686202
Name: _FracPlacementsRenewedByClient, Length: 1348, dtype: float64

In [19]:
data['_ChurnProbScore'] = churn_prob_score(data)
data.head()

d:\TejaMeruva\ApexRenew\prioritization.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.PlacementClientSegmentCode.replace(pd.NA, 'null', inplace=True)
d:\TejaMeruva\ApexRenew\prioritization.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.IncumbentIndicator.replace(pd.NA, 'null', inplace=True)


,Client,PlacementClientLocalID,PlacementName,Coverage,ProductLine,CarrierGroup,PlacementCreatedDate/Time,PlacementCreatedBy,PlacementCreatedBy_ID,ResponseReceivedDate,PlacementSpecialist,PlacementRenewingStatus,PlacementStatus,DeclinationReason,PlacementId,PlacementEffectiveDate,PlacementExpiryDate,IncumbentIndicator,ParticipationStatusCode,PlacementClientSegmentCode,PlacementRenewingStatusCode,Limit,CoveragePremiumAmount,TriaPremium,TotalPremium,Comission%,ComissionAmount,ParticipationPercentage,CarrierGroupLocalID,ProductionCode,SubmissionSentDate,ProgramProductLocalCodeText,ApproachNonAdmittedMarketIndicator,CarrierIntegration,_DaysToExpiry,_CarrierResponseTime,_ChurnStatus,_FracPlacementsRenewedByClient,_FracPlacementsRenewedByCarrier,_ChurnProbScore
0,Global Technologies,SCR-0b810b6f4c20,SCR-8d9f15ee3a3c,General Liability,Energy and Power,Eastern Risk Management,2025-04-24T06:37:09.314837765,Kimberly Jackson,SCR-c54656cdfecb,2025-07-29,Mary Jackson,In progress,Quote,<NA>,SCR-76fd0b40a1cb,2025-09-30,2026-09-30,N,QUOTATION_STATUS_QUOTED,CLIENT_SEGMENT_RISK_MGMT,RENEWAL_STATUS_IN_PROGRESS,3558700.0,65304.28,1881.62,67311.79,8,5760.41,100,0498,PRODUCTION_TYPE_NEW,NaT,SCR-262eac00ad8f,N,Not Applicable,365,NaN,C,0.000000,0.477273,0.411721
1,Apex Enterprises,SCR-cfcd65ae0f7a,SCR-6b2e08c3f11d,General Liability,Casualty,Liberty Insurance Group,2025-04-25T15:21:40.575782415,Matthew Johnson,SCR-d47a7dd7c8ff,2025-07-29,Donald Martin,In progress,Quote,<NA>,SCR-07c81694713d,2025-09-15,2026-09-15,Y,QUOTATION_STATUS_QUOTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,82254400.0,1441557.71,55830.17,1581997.56,15,282637.79,100,0815,PRODUCTION_TYPE_RENEWAL,NaT,SCR-af66d3bd1f7b,Y,Not Applicable,365,NaN,C,0.480000,0.484444,0.756677
2,Omega Global,SCR-51dd14cf0f45,SCR-6d2b42de057e,General Liability,Casualty,United Coverage Corp,2025-03-13T18:03:34.574937777,Robert Young,SCR-0b51d24e469e,NaT,Michelle Anderson,In progress,Submitted,<NA>,SCR-b422393095aa,2025-09-15,2026-09-15,Y,QUOTATION_STATUS_SUBMITTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,225000.0,3279.01,0.00,3256.21,17,645.63,100,3093,PRODUCTION_TYPE_RENEWAL,2025-07-29,SCR-af66d3bd1f7b,N,Not Applicable,365,NaN,C,0.214286,0.478673,0.556380
3,Omega Global,SCR-51dd14cf0f45,SCR-6d2b42de057e,General Liability,Casualty,United Coverage Corp,2025-03-13T18:03:34.574937777,Robert Young,SCR-0b51d24e469e,NaT,Michelle Anderson,In progress,Submitted,<NA>,SCR-b422393095aa,2025-09-15,2026-09-15,Y,QUOTATION_STATUS_SUBMITTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,225000.0,0.00,0.00,0.00,17,0.00,100,3093,PRODUCTION_TYPE_RENEWAL,2025-07-29,SCR-af66d3bd1f7b,N,Not Applicable,365,NaN,C,0.214286,0.478673,0.304154
4,Omega Global,SCR-51dd14cf0f45,SCR-6d2b42de057e,General Liability,Casualty,Northern Trust Insurance,2025-03-13T18:03:34.574937777,Robert Young,SCR-0b51d24e469e,NaT,Michelle Anderson,In progress,Submitted,<NA>,SCR-b422393095aa,2025-09-15,2026-09-15,N,QUOTATION_STATUS_SUBMITTED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,225000.0,0.00,0.00,0.00,17,0.00,100,4584,PRODUCTION_TYPE_RENEWAL,2025-07-30,SCR-af66d3bd1f7b,N,Not Applicable,365,NaN,C,0.214286,0.459016,0.344955


In [20]:
data.sort_values(['_ChurnProbScore'], ascending=False).head(3)

,Client,PlacementClientLocalID,PlacementName,Coverage,ProductLine,CarrierGroup,PlacementCreatedDate/Time,PlacementCreatedBy,PlacementCreatedBy_ID,ResponseReceivedDate,PlacementSpecialist,PlacementRenewingStatus,PlacementStatus,DeclinationReason,PlacementId,PlacementEffectiveDate,PlacementExpiryDate,IncumbentIndicator,ParticipationStatusCode,PlacementClientSegmentCode,PlacementRenewingStatusCode,Limit,CoveragePremiumAmount,TriaPremium,TotalPremium,Comission%,ComissionAmount,ParticipationPercentage,CarrierGroupLocalID,ProductionCode,SubmissionSentDate,ProgramProductLocalCodeText,ApproachNonAdmittedMarketIndicator,CarrierIntegration,_DaysToExpiry,_CarrierResponseTime,_ChurnStatus,_FracPlacementsRenewedByClient,_FracPlacementsRenewedByCarrier,_ChurnProbScore
231,Zenith Global,SCR-6bbe5752fdd3,SCR-92e63acf245b,General Liability,Energy and Power,American Risk Partners,2025-05-28T16:45:15.512852722,David Lee,SCR-363a9e8d56b1,2025-05-30,Margaret White,In progress,Received,<NA>,SCR-6ed4b3aca26e,2025-04-30,2026-04-30,N,QUOTATION_STATUS_BINDER_RECEIVED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,NaN,22342.75,0.0,23350.75,17,4700.33,100,5261,PRODUCTION_TYPE_NEW,NaT,SCR-262eac00ad8f,Y,Not Applicable,365,NaN,N,0.5,0.461538,9.992582
318,Crown Enterprises,SCR-35a2638ad599,SCR-a46dc1539071,General Liability,Energy and Power,Shield Insurance Alliance,2025-01-08T16:11:28.288558541,Paul Garcia,SCR-02694c5520f1,2025-03-03,Margaret White,In progress,Received,<NA>,SCR-3354b984a9fd,2025-03-01,2026-03-01,N,QUOTATION_STATUS_BINDER_RECEIVED,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_IN_PROGRESS,NaN,27447.64,0.0,22477.06,18,4147.53,100,B307,PRODUCTION_TYPE_RENEWAL,NaT,SCR-262eac00ad8f,Y,Not Applicable,365,NaN,N,0.5,0.500000,9.985163
781,Premier Group,SCR-fe30bdee0d25,SCR-09309fb0e27c,General Liability,Casualty,Shield Insurance Alliance,2024-03-14T18:16:28.479,Joseph Miller,SCR-cb511e017475,2024-03-14,Mary Wilson,Renewed (Full),Not Taken Up,<NA>,SCR-08f2c5072700,2024-04-01,2025-04-01,N,QUOTATION_STATUS_NTUD,CLIENT_SEGMENT_MIDDLE_MKT,RENEWAL_STATUS_FULLY_RENEWED,823440.0,18599.86,0.0,23405.87,18,4187.38,100,B307,PRODUCTION_TYPE_NEW,NaT,SCR-af66d3bd1f7b,Y,Not Applicable,365,NaN,N,0.5,0.500000,9.977745


In [21]:
from prioritization import client_priority_GPA

client_priority_GPA(data)

d:\TejaMeruva\ApexRenew\prioritization.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.PlacementClientSegmentCode.replace(pd.NA, 'null', inplace=True)
d:\TejaMeruva\ApexRenew\prioritization.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X.IncumbentIndicator.replace(pd.NA, 'null', inplace=True)


0       0.617812
1       1.301362
2       0.791416
3       0.583050
4       0.566750
          ...   
1343    7.019007
1344    4.598068
1345    5.644295
1346    7.506572
1347    7.506572
Length: 1348, dtype: float64

## Misc

In [23]:
data.columns

Index(['Client', 'PlacementClientLocalID', 'PlacementName', 'Coverage',
       'ProductLine', 'CarrierGroup', 'PlacementCreatedDate/Time',
       'PlacementCreatedBy', 'PlacementCreatedBy_ID', 'ResponseReceivedDate',
       'PlacementSpecialist', 'PlacementRenewingStatus', 'PlacementStatus',
       'DeclinationReason', 'PlacementId', 'PlacementEffectiveDate',
       'PlacementExpiryDate', 'IncumbentIndicator', 'ParticipationStatusCode',
       'PlacementClientSegmentCode', 'PlacementRenewingStatusCode', 'Limit',
       'CoveragePremiumAmount', 'TriaPremium', 'TotalPremium', 'Comission%',
       'ComissionAmount', 'ParticipationPercentage', 'CarrierGroupLocalID',
       'ProductionCode', 'SubmissionSentDate', 'ProgramProductLocalCodeText',
       'ApproachNonAdmittedMarketIndicator', 'CarrierIntegration',
       '_DaysToExpiry', '_CarrierResponseTime', '_ChurnStatus',
       '_FracPlacementsRenewedByClient', '_FracPlacementsRenewedByCarrier',
       '_ChurnProbScore'],
      dtype='obje